In [2]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 
### NOTE:  Major change due to conversion of some bank holding companies back to bank structures.
###        (This removes the Parent (NAMEHCR) and ParRSSD (RSSDHCR) fields from the institutions2.zip listing.)
###        

import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [3]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201903.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2019-03-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2019-03-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2019-03-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2019-03-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.493106e+09  1.467718e+09  5.250193e+08  6.918614e+08   
1 2001-03-31  1.276944e+09  1.280503e+09  6.684335e+08  8.181268e+08   
2 2001-03-31  1.068269e+09  1.065448e+09  5.102168e+08  2.889343e+08   
3 2001-03-31  9.033065e+08  8.918872e+08  5.431489e+08  6.294408e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.545860e+08  9.464474e+08        NaT  9.210716   9.210716  5.683020   
1  1.050719e+08  9.231987e+08 2009-01-16  7.877250  17.087966  7.235392   
2  4.426447e+08  7.315790e+08 2008-11-23  6.589967  23.677933  5.522791   
3  6.712000e+07  6.965608e+08        NaT  5.572343  29.250276  5.879261   
4  2.469325e+07  2.951088e+08        NaT  2.724441  31.974717  2.756292   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.683020  7.488985   7.488985  
1  12.918411  8.855732  16.344717  
2  18.441202  3.127540  19.472258  
3  24.320463  6.813320  26.285578  
4  27.076756  2.927087  29.212664

In [4]:
len(rk) # 10776

10776

In [5]:
bhc = pd.read_csv("INSTITUTIONS2_201903.csv", encoding='latin-1') # Warning: This is latest available file as of 2019-April.
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street     48,570      NM   
1  Connecticut     6       0       0  81 West Main Street    624,655      NM   
2        Maine     8       0       0      One City Center  1,699,404      SM   
3        Maine     9    9781       0       66 Main Street    539,169      SM   
4        Maine    10       0       0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3  ...  \
0       223         0         0  ...   
1       223       999         0  ...   
2       223         0         0  ...   
3       223         0         0  ...   
4       223         0         0  ...   

                                       CBSA  CBSA_NO  \
0  Hartford-West Hartford-East Hartford, CT    25540   
1                     New Haven-Milford, CT    35300   
2               Portland-South Portland, ME    38860   
3                                       NaN        0   
4               Portland-South Portland, ME    38860   

                            CBSA_METRO_NAME CBSA_METRO CBSA_METRO_FLG  \
0  Hartford-West Hartford-East Hartford, CT      25540              1   
1                     New Haven-Milford, CT      35300              1   
2               Portland-South Portland, ME      38860              1   
3                                       NaN          0              0   
4               Portland-South Portland, ME      38860              1   

  CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              0       NaN            0            0 NaN  
1              0       NaN            0            0 NaN  
2              0       NaN            0            0 NaN  
3              0       NaN            0            0 NaN  
4              0       NaN            0            0 NaN  

[5 rows x 133 columns]

In [6]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [7]:
len(bhc) # 27607

27607

In [8]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street   48,570      NM   
1  Connecticut     6       0       0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3  ...  \
0       223         0         0  ...   
1       223       999         0  ...   

                                       CBSA  CBSA_NO  \
0  Hartford-West Hartford-East Hartford, CT    25540   
1                     New Haven-Milford, CT    35300   

                            CBSA_METRO_NAME CBSA_METRO CBSA_METRO_FLG  \
0  Hartford-West Hartford-East Hartford, CT      25540              1   
1                     New Haven-Milford, CT      35300              1   

  CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0              0       NaN            0            0 NaN  
1              0       NaN            0            0 NaN  

[2 rows x 133 columns]

In [9]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
#bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
bhc2.rename(index=str, columns={"CERT":"Cert", "FED_RSSD":"IDRSSD", "NAME":"Name", "STALP":"ST", "BKCLASS":"BkClass", \
            "OFFICES":"Offices", "NAMEHCR":"Parent", "RSSDHCR":"ParRSSD", "STALPHCR":"ParST", "ULTCERT":"UltCert"}, inplace=True)
len(bhc2) # 27607

27607

In [10]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [11]:
bhc2.columns

Index(['Cert', 'IDRSSD', 'Name', 'ST', 'BkClass', 'Offices', 'Parent',
       'ParRSSD', 'ParST', 'UltCert'],
      dtype='object')

In [12]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635                           AuburnBank  AL      SM      8.0   
19    39  430036            Robertson Banking Company  AL      NM      7.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      3.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA       14  
15        Auburn National Bancorp Inc  1129533.0    AL       35  
19                            Rbc Inc  2526755.0    AL       39  
21       Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Fin Services Inc  1082955.0    AL       46  
26   Traders & Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares IncESOPlan  3587294.0    AL       50  
30        First Vernon Bancshares Inc  1139998.0    AL       51  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL       52  
33                       Gadsden Corp  1079067.0    AL       54

In [13]:
#bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

In [14]:
#bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

In [16]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201812.csv", index_col=0)
#rkhn = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612n.csv", index_col=0)

#len(rkh), len(rkhn) # 10762
len(rkh) # 10775

10775

In [17]:
len(rkh[rkh['Ticker']>'']) # 3143

3143

In [18]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 8765

8765

In [19]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10771  10772  1094314.0    MO  CBCYB
10774  10775  3447567.0    TX    NaN

In [20]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 404

404

In [21]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [22]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [23]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACBI          1
ACFC          1

In [24]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [25]:
rkh[rkh['Ticker']=="BHWB"].iloc[:,0:12] # check on Ticker that was misidentified in earlier runs (should be 2 banks only)

Rank  IDRSSD       xAsOf           xName xState    xCert       nAsOf  \
1736  1737   27548  2018-12-31  BLACKHAWK BANK     WI  14078.0  2001-03-31   
7737  7738  388931  2003-09-30   FIRST BANK BC     IL  16551.0  2001-03-31   

     FailDate BkClass  Offices                 Parent    ParRSSD  
1736      NaN      NM     10.0  Blackhawk Bancorp Inc  1491913.0  
7737      NaN      NM      4.0  Blackhawk Bancorp Inc  1491913.0

In [26]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [27]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4363.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [28]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4363.0  Bank Of America Corp  1073757.0    NC     3510

In [29]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2019-03-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2019-03-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...   pr_eTLcs    pr_eDD  \
0  1.493106e+09  1.467718e+09  5.250193e+08  ...   5.683020  7.488985   
1  1.276944e+09  1.280503e+09  6.684335e+08  ...  12.918411  8.855732   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.488985   628.0        N   5121.0  Jpmorgan Chase & Co.  1039502.0   
1  16.344717  3510.0        N   4363.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [30]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [31]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2019-03-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2019-03-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...    pr_eDD   pr_eDDcs    Cert  \
0  1.493106e+09  1.467718e+09  5.250193e+08  ...  7.488985   7.488985   628.0   
1  1.276944e+09  1.280503e+09  6.684335e+08  ...  8.855732  16.344717  3510.0   

  BkClass  Offices                Parent    ParRSSD  ParST  UltCert  Ticker  
0       N   5121.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0     JPM  
1       N   4363.0  Bank Of America Corp  1073757.0     NC   3510.0     BAC  

[2 rows x 28 columns]

In [32]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [33]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2019-03-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2019-03-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices  ...       aeTotLn        aeDDep        aeFDep  \
0        NaT       N   5121.0  ...  5.250193e+08  6.918614e+08  2.545860e+08   
1 2009-01-16       N   4363.0  ...  6.684335e+08  8.181268e+08  1.050719e+08   

       aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD   pr_eDDcs  
0  9.464474e+08  9.210716   9.210716  5.683020   5.683020  7.488985   7.488985  
1  9.231987e+08  7.877250  17.087966  7.235392  12.918411  8.855732  16.344717  

[2 rows x 27 columns]

In [ ]:
## New code (May 2019) to add holding company values for Parent, ParRSSD, Ticker for Zions Bancorp (since this data)
## was recently removed due to legal changes to terminate Fed CCAR oversight
## If other companies do the same, their data will be added back here
## (This is essential for the historical CCAR 31-bank group to compute properly)

In [34]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent  ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0    NaN    NaN      0.0

In [35]:
rk4.loc[rk4.xCert == 2270, 'Parent'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'Ticker'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'ParRSSD'] = 1027004.0

In [36]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [37]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201903.csv")

In [38]:
len(rk4) # 10776

10776

In [39]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 206

206

In [40]:
rkm.head()

Rank  IDRSSD      xAsOf                         xName xState    xCert  \
79     80   63069 2019-03-31            CITY NATIONAL BANK     CA  17281.0   
143   144  527954 2019-03-31  FIRST NATIONAL BANK OF OMAHA     NE   5452.0   
149   150  311845 2019-03-31                   ARVEST BANK     AR   8728.0   
174   175  877369 2019-03-31                  RABOBANK, NA     CA  23364.0   
175   176  137915 2019-03-31                 EMIGRANT BANK     NY  12054.0   

         nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
79  2001-03-31      NaT       N     77.0  ...  1.563641e+07  2.066361e+07   
143 2001-03-31      NaT       N    126.0  ...  8.036021e+06  9.398646e+06   
149 2001-03-31      NaT      SM    298.0  ...  7.044136e+06  9.349540e+06   
174 2001-03-31      NaT       N    107.0  ...  6.225848e+06  6.235317e+06   
175 2001-03-31      NaT      SB      5.0  ...  5.087693e+06  6.181834e+06   

          aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
79   1702.082192  2.066531e+07  0.148244  67.321373  0.169255  62.709636   
143     0.000000  9.398646e+06  0.072589  73.877334  0.086985  69.904138   
149  3152.657534  9.352693e+06  0.068349  74.304410  0.076249  70.375292   
174     0.000000  6.235317e+06  0.054885  75.824522  0.067391  72.051738   
175     0.000000  6.181834e+06  0.054669  75.879190  0.055071  72.106809   

       pr_eDD   pr_eDDcs  
79   0.223671  66.543839  
143  0.101735  74.196128  
149  0.101203  74.698565  
174  0.067494  76.618728  
175  0.066915  76.685642  

[5 rows x 27 columns]

In [41]:
rkm.to_csv('rk_missingtickers201903.csv')

In [42]:
##############################################################################

In [43]:
## start up here after filling in missing tickers

In [44]:
## filled in many missing tickers

In [45]:
rkm2 = pd.read_csv('rk_missingtickers201903a.csv',index_col=0)
rkm2.head()

Rank  IDRSSD      xAsOf                         xName xState  xCert  \
79     80   63069  3/31/2019            CITY NATIONAL BANK     CA  17281   
143   144  527954  3/31/2019  FIRST NATIONAL BANK OF OMAHA     NE   5452   
149   150  311845  3/31/2019                   ARVEST BANK     AR   8728   
174   175  877369  3/31/2019                  RABOBANK, NA     CA  23364   
175   176  137915  3/31/2019                 EMIGRANT BANK     NY  12054   

         nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
79   3/31/2001      NaN       N       77  ...  1.563641e+07  2.066361e+07   
143  3/31/2001      NaN       N      126  ...  8.036021e+06  9.398646e+06   
149  3/31/2001      NaN      SM      298  ...  7.044136e+06  9.349540e+06   
174  3/31/2001      NaN       N      107  ...  6.225848e+06  6.235317e+06   
175  3/31/2001      NaN      SB        5  ...  5.087693e+06  6.181834e+06   

          aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
79   1702.082192  2.066531e+07  0.148244  67.321373  0.169255  62.709636   
143     0.000000  9.398646e+06  0.072589  73.877334  0.086985  69.904138   
149  3152.657534  9.352693e+06  0.068349  74.304410  0.076249  70.375292   
174     0.000000  6.235317e+06  0.054885  75.824522  0.067391  72.051738   
175     0.000000  6.181834e+06  0.054669  75.879190  0.055071  72.106809   

       pr_eDD   pr_eDDcs  
79   0.223671  66.543839  
143  0.101735  74.196128  
149  0.101203  74.698565  
174  0.067494  76.618728  
175  0.066915  76.685642  

[5 rows x 27 columns]

In [46]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
79   5280254     RY
192  4882802   CADE
286  3814208    LBC
325  2609975   OCFC
460  5280254     RY

In [47]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 15, 13

(15, 13)

In [48]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName                          Parent Ticker
79                CITY NATIONAL BANK       Rbc Us Group Holdings Llc    NaN
192               CADENCE BANK, N.A.                 Cadence Bancorp    NaN
286           LUTHER BURBANK SAVINGS             Luther Burbank Corp    NaN
325              OCEANFIRST BANK, NA            Oceanfirst Fin Corp.    NaN
460           RBC BANK (GEORGIA), NA       Rbc Us Group Holdings Llc    NaN
507                        FIRSTBANK                     Fb Fin Corp    NaN
546                      ORIGIN BANK              Origin Bancorp Inc    NaN
675               CADENCE BANK, N.A.                 Cadence Bancorp    NaN
928         LIVE OAK BANKING COMPANY         Live Oak Bancshares Inc    NaN
949          PACIFIC MERCANTILE BANK      Pacific Mercantile Bancorp    NaN
1012                  RED RIVER BANK        Red River Bancshares Inc    NaN
1145          VERITEX COMMUNITY BANK            Veritex Holdings Inc    NaN
1209                   SOUTHERN BANK   Southern Missouri Bancorp Inc    NaN
1252  FINEMARK NATIONAL BANK & TRUST           Finemark Holdings Inc    NaN
1315   CITIZENS COMMUNITY FEDERAL NA  Citizens Community Bancorp Inc    NaN
2666             FIRST GULF BANK, NA       Rbc Us Group Holdings Llc    NaN

In [49]:
rk4c = rk4.copy()

In [50]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{5280254: 'RY',
 4882802: 'CADE',
 3814208: 'LBC',
 2609975: 'OCFC',
 1132104: 'FBK',
 1885307: 'OBNK',
 3884863: 'LOB',
 2869733: 'PMBC',
 2743235: 'RRBI',
 4105266: 'VBTX',
 3266227: 'SMBC',
 3547159: 'FNBT',
 3823929: 'CZWI'}

In [51]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [53]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','ParRSSD','Ticker']]

xName                          Parent  \
79                CITY NATIONAL BANK       Rbc Us Group Holdings Llc   
192               CADENCE BANK, N.A.                 Cadence Bancorp   
286           LUTHER BURBANK SAVINGS             Luther Burbank Corp   
325              OCEANFIRST BANK, NA            Oceanfirst Fin Corp.   
460           RBC BANK (GEORGIA), NA       Rbc Us Group Holdings Llc   
507                        FIRSTBANK                     Fb Fin Corp   
546                      ORIGIN BANK              Origin Bancorp Inc   
675               CADENCE BANK, N.A.                 Cadence Bancorp   
928         LIVE OAK BANKING COMPANY         Live Oak Bancshares Inc   
949          PACIFIC MERCANTILE BANK      Pacific Mercantile Bancorp   
1012                  RED RIVER BANK        Red River Bancshares Inc   
1145          VERITEX COMMUNITY BANK            Veritex Holdings Inc   
1209                   SOUTHERN BANK   Southern Missouri Bancorp Inc   
1252  FINEMARK NATIONAL BANK & TRUST           Finemark Holdings Inc   
1315   CITIZENS COMMUNITY FEDERAL NA  Citizens Community Bancorp Inc   
2666             FIRST GULF BANK, NA       Rbc Us Group Holdings Llc   

        ParRSSD Ticker  
79    5280254.0     RY  
192   4882802.0   CADE  
286   3814208.0    LBC  
325   2609975.0   OCFC  
460   5280254.0     RY  
507   1132104.0    FBK  
546   1885307.0   OBNK  
675   4882802.0   CADE  
928   3884863.0    LOB  
949   2869733.0   PMBC  
1012  2743235.0   RRBI  
1145  4105266.0   VBTX  
1209  3266227.0   SMBC  
1252  3547159.0   FNBT  
1315  3823929.0   CZWI  
2666  5280254.0     RY

In [54]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 518, 509

(511, 495)

In [55]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201903.csv")

In [56]:
## process ultcert version:

In [57]:
rk5 = rk4.copy()

In [58]:
ult = set(rk5['UltCert'])
len(ult) # 5703

5703

In [59]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [60]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4183

4183

In [61]:
len(bhcc[bhcc['Ticker']>'']) # 501

501

In [62]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [63]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
39     16571.0                 Bank Of Montreal    BMO  1231333.0     0
192     4999.0                  Cadence Bancorp   CADE  4882802.0    TX
256    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
460    26342.0        Rbc Us Group Holdings Llc     RY  5280254.0     0
488    35385.0        United Community Fin Corp   UCFC  3831250.0    OH
675     4999.0                  Cadence Bancorp   CADE  4882802.0    TX
697    35525.0  Atlantic Capital Bancshares Inc   ACBI  3555686.0    GA
997    35525.0  Atlantic Capital Bancshares Inc   ACBI  3555686.0    GA
1073    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1193   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1596    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
1718   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1784      89.0   White River Bancshares Company    NaN  3306589.0    AR
1792   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1968   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
2189   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2666   26342.0        Rbc Us Group Holdings Llc     RY  5280254.0     0
2895   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4054   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4239   35385.0        United Community Fin Corp   UCFC  3831250.0    OH
4351   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4454   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
4625   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
4907    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
5970    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6798   17399.0                   First Fin Corp    NaN  1067514.0    KS
7099      89.0   White River Bancshares Company    NaN  3306589.0    AR
7105   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
7429   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7502   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7763    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
8773   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
9116   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9672    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9791   17399.0                   First Fin Corp    NaN  1067514.0    KS
10197  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10233  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10521  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [64]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6798  17399.0  First Fin Corp    NaN  1067514.0    KS
9791  17399.0  First Fin Corp    NaN  1067514.0    KS

In [65]:
len(bhcc) # 4183

4183

In [66]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4145

4145

In [67]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [68]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [69]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [70]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

'Jpmorgan Chase & Co.'

In [71]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [72]:
len(rk5c[rk5c['Ticker']>'']) # 3266

3266

In [73]:
j # 4563

4563

In [74]:
len(rk5c) # 10776

10776

In [75]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult201903.csv")

In [76]:
# I use ZION to show how many historic banks and mergers got collapsed into one bank charter

In [77]:
rk4.loc[rk4.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] # current bank

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [78]:
rk5c.loc[rk5c.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] #historic banks merged in Zions 

IDRSSD                                 xName    xCert Ticker Parent  \
78     276579              ZIONS BANCORPORATION, NA   2270.0   ZION   ZION   
154    837260               CALIFORNIA BANK & TRUST  20852.0   ZION   ZION   
161    676656                        AMEGY BANK, NA  24107.0   ZION   ZION   
309   1004368              NATIONAL BANK OF ARIZONA  20626.0   ZION   ZION   
336    456960                     NEVADA STATE BANK  18113.0   ZION   ZION   
437    933957              VECTRA BANK COLORADO, NA   2993.0   ZION   ZION   
719    807665                     VINEYARD BANK, NA  23556.0   ZION   ZION   
1107   458263                  STOCKMEN'S BANK, THE  23182.0   ZION   ZION   
1148  1198641  COMMERCE BANK OF WASHINGTON, NA, THE  27298.0   ZION   ZION   
1221  2479310                     SILVER STATE BANK  34194.0   ZION   ZION   
1644   720465                         ALLIANCE BANK  23124.0   ZION   ZION   
1716   618656                            KLEIN BANK  22866.0   ZION   ZION   
2748   105950          MINNEQUA BANK OF PUEBLO, THE   1773.0   ZION   ZION   
3227   576158                             MAXIMBANK   9017.0   ZION   ZION   
4121   309664                           RANCHO BANK  22934.0   ZION   ZION   
4228   521550                        LONE STAR BANK  22304.0   ZION   ZION   
4700  2068134            GREAT BASIN BANK OF NEVADA  33824.0   ZION   ZION   
6224   626754              LEAGUE CITY BANK & TRUST  17577.0   ZION   ZION   
6431  3161993                           CHOICE BANK  57469.0   ZION   ZION   
6712   687269        INTERCONTINENTAL NATIONAL BANK  19112.0   ZION   ZION   
6747   456670                   FRONTIER STATE BANK  24227.0   ZION   ZION   
8435  3374186          COMMERCE BANK OF OREGON, THE  58223.0   ZION   ZION   
9909  2907448               SOUTHLAND BUSINESS BANK  35287.0   ZION   ZION   

        ParRSSD  
78    1027004.0  
154   1027004.0  
161   1027004.0  
309   1027004.0  
336   1027004.0  
437   1027004.0  
719   1027004.0  
1107  1027004.0  
1148  1027004.0  
1221  1027004.0  
1644  1027004.0  
1716  1027004.0  
2748  1027004.0  
3227  1027004.0  
4121  1027004.0  
4228  1027004.0  
4700  1027004.0  
6224  1027004.0  
6431  1027004.0  
6712  1027004.0  
6747  1027004.0  
8435  1027004.0  
9909  1027004.0